this multilogistic regression finds inspiration in <https://runawayhorse001.github.io/LearningApacheSpark/classification.html>

begins at 11.2. Multinomial logistic regression

<b> to do: create a pipeline => better for MLflow </b>

In [0]:
import sys
sys.version_info

Out[1]: sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)

# Training the model

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.dbutils import DBUtils
import numpy as np
import pandas as pd
# enables use in dbutils
dbutils = DBUtils(spark)
# display datframes
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark.conf.set("spark.databricks.io.cache.enabled", True) # delta caching
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", True) # adaptive query execution for skewed data
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10485760) # setting treshhold on broadcasting 
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true") # conversion between pyspark and pandas dataframe
spark.conf.set("spark.databricks.optimizer.rangeJoin.binSize", 1000) #range optimizer

In [0]:
#load in he necessary function
from pyspark.sql.functions import isnan, when, count, col
#data preprocessen 
df_train  = spark.read.format('delta').load(
 "dbfs:/mnt/bastore/modelling/recommendations/data_for_model/train")
df_train.count()



Out[3]: 23096

In [0]:

df_train = df_train.select("service", "Country","Seniority_max", "Product_sum", "Upsell_avg", "SubBreak_sum", "Retention_sum", "RetIntention_sum", "Discount_sum", "Payments_sum", "Fline", "Hline", "ExHline", "ExFline", "Seniority_Hline", "Seniority_Fline", "Seniority_ExFline", "Seniority_ExHline")
# drop nan/null values

df_train1 = df_train.na.drop(how="any")

df_train1.groupBy('service').count()

Out[4]:

service,count
PAS,6089
Freebook,1485
Online,15522


In [0]:
#create a balanced dataset
newdata1 = df_train1.filter((df_train1.service == "Online")|(df_train1.service == "Freebook"))
newdata2 = df_train1.filter((df_train1.service == "PAS") | (df_train1.service == "Freebook"))

X1 = newdata1.drop("service")
Y1 = newdata1.select("service")
X2 = newdata2.drop("service")
Y2 = newdata2.select("service")



In [0]:
# resample(newdata2, 1, "service", "Freebook")

pos = newdata1.filter(col('service') == "Freebook")
neg = newdata1.filter(col('service') != "Freebook")
total_pos = pos.count()
total_neg = neg.count()
fraction=float(total_pos*1)/float(total_neg)
# data_balanced_over1

In [0]:
from pyspark.sql.functions import col
def resample(base_features,ratio,class_field,base_class):
    pos = base_features.filter(col(class_field)==base_class)
    neg = base_features.filter(col(class_field)!=base_class)
    total_pos = pos.count()
    total_neg = neg.count()
    fraction=float(total_pos*ratio)/float(total_neg)
    sampled = neg.sample(False,fraction)
    return sampled.union(pos)
 
data_balanced_over1 = resample(newdata1, 1, "service", "Freebook")
data_balanced_over2 = resample(newdata2, 1, "service", "Freebook")

newdata3 = data_balanced_over1.filter(col("service") == "Online")

Train = newdata3.union(data_balanced_over2)

In [0]:
#index country
from pyspark.ml.feature import IndexToString,StringIndexer, VectorIndexer


featureIndexer = StringIndexer(inputCol="Country", outputCol="CountryIndex").fit(Train)

Train1 = featureIndexer.transform(Train).drop('Country')

features need to be in a vector

In [0]:
#option 1 following website
#transform dataset to dataframe
from pyspark.ml.linalg import Vectors # !!!!caution: not from pyspark.mllib.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
def transData(data): 
  return data.rdd.map(lambda r: [Vectors.dense(r[1:]),r[0]]).toDF(['features', 'label'])

In [0]:
Train2 =transData(Train1)
Train2.select("features", "label").show(10, False)

+-------------------------------------------------------------------------+------+
features |label |
+-------------------------------------------------------------------------+------+
[15.0,1.0,0.0,1.0,0.0,0.0,0.0,134.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,1.0]|Online|
[15.0,4.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,1.0] |Online|
[15.0,3.0,0.0,1.0,0.0,2.0,0.0,119.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,1.0]|Online|
[15.0,3.0,0.0,0.0,0.0,0.0,0.0,114.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,1.0]|Online|
[15.0,3.0,0.0,0.0,0.0,1.0,0.0,138.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0]|Online|
[15.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0] |Online|
[15.0,1.0,0.0,1.0,0.0,0.0,0.0,58.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.0,3.0] |Online|
[15.0,3.0,0.0,0.0,0.0,0.0,0.0,143.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,3.0]|Online|
[15.0,4.0,0.0,0.0,0.0,0.0,0.0,259.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,28.0,3.0]|Online|
[15.0,3.0,0.0,1.0,0.0,1.0,0.0,137.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0]|Online|
+-------------------------------------------------------------------------+------+
only showing top 10 rows

In [0]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol='label',
                             outputCol='indexedLabel').fit(Train2)
Train3=labelIndexer.transform(Train2)



In [0]:
from pyspark.ml.classification import LogisticRegression
import numpy as np
from scipy.sparse import csr_matrix
logr = LogisticRegression(featuresCol='features', labelCol='indexedLabel')

In [0]:
# Convert indexed labels back to original labels.
#labelConverter = IndexToString(inputCol="prediction", #outputCol="predictedLabel", labels=labelIndexer.labels)
#Train4 = labelConverter.transform(Train3)
#Train4.show()

In [0]:
model = logr.fit(Train3)

In [0]:
model.explainParam('labelCol')

Out[17]: 'labelCol: label column name. (default: label, current: indexedLabel)'

# model evaluation

In [0]:
print(model.coefficientMatrix)

DenseMatrix([[ 0.00000000e+00, 9.41704331e-02, 0.00000000e+00,
 -7.91784436e-02, -3.27839677e-01, -1.45620671e-01,
 1.42662330e-02, -6.97674492e-03, -2.07639801e-01,
 -4.65710162e-01, -3.69343278e-01, -1.54162716e-01,
 -2.99711357e-02, 1.12357490e-01, -2.04705657e-02,
 2.86003146e-03, 2.11079215e-01],
 [ 0.00000000e+00, -5.90462706e-02, 0.00000000e+00,
 1.32468129e-01, 3.52697792e-01, 4.93188182e-02,
 -2.44524917e-02, 5.52551118e-03, -6.67401061e-02,
 1.12366716e+00, 3.75744742e-01, 3.74507266e-01,
 -1.81333614e-02, -3.67828979e-02, 1.17396223e-02,
 1.72343814e-04, -6.38191028e-01],
 [ 0.00000000e+00, -3.51241626e-02, 0.00000000e+00,
 -5.32896851e-02, -2.48581154e-02, 9.63018530e-02,
 1.01862587e-02, 1.45123374e-03, 2.74379907e-01,
 -6.57956997e-01, -6.40146344e-03, -2.20344551e-01,
 4.81044971e-02, -7.55745926e-02, 8.73094336e-03,
 -3.03237527e-03, 4.27111813e-01]])

In [0]:
trainingSummary = model.summary

In [0]:
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

False positive rate by label:
label 0: 0.17882592466915506
label 1: 0.23483827493261455
label 2: 0.17352056168505517
True positive rate by label:
label 0: 0.5584329349269588
label 1: 0.7872053872053872
label 2: 0.478796169630643
Precision by label:
label 0: 0.6147660818713451
label 1: 0.6264737406216506
label 2: 0.5742411812961444
Recall by label:
label 0: 0.5584329349269588
label 1: 0.7872053872053872
label 2: 0.478796169630643
F-measure by label:
label 0: 0.5852470424495476
label 1: 0.6977021784541928
label 2: 0.5221932114882507

In [0]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.6085784864136538
FPR: 0.1957632477004785
TPR: 0.6085784864136536
F-measure: 0.602047216736878
Precision: 0.6053653337472176
Recall: 0.6085784864136536

In [0]:
predictions = model.transform(Train3)
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
prediction = labelConverter.transform(predictions)
prediction.show(5, False)

+-------------------------------------------------------------------------+------+------------+---------------------------------------------------------------+------------------------------------------------------------+----------+--------------+
features |label |indexedLabel|rawPrediction |probability |prediction|predictedLabel|
+-------------------------------------------------------------------------+------+------------+---------------------------------------------------------------+------------------------------------------------------------+----------+--------------+
[15.0,1.0,0.0,1.0,0.0,0.0,0.0,134.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,1.0]|Online|0.0 |[1.8942204836165142,-1.0799867658734392,-0.8142337177430748] |[0.8946723238168475,0.04570694625179002,0.05962072993136241]|0.0 |Online |
[15.0,4.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,1.0] |Online|0.0 |[0.40709657772869734,-0.35043326685200726,-0.05666331087669019]|[0.4767040431660043,0.22348985000341853,0.2998061068305771] |0.0 |Online |
[15.0,3.0,0.0,1.0,0.0,2.0,0.0,119.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,1.0]|Online|0.0 |[-0.49702857150230106,0.28782597342188987,0.20920259808041103] |[0.19162859061336956,0.4200667468192428,0.3883046625673877] |1.0 |Freebook |
[15.0,3.0,0.0,0.0,0.0,0.0,0.0,114.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,1.0]|Online|0.0 |[-0.09172506091303334,0.02909265234360603,0.06263240856942759] |[0.3034600075535396,0.3424300982751064,0.3541098941713541] |2.0 |PAS |
[15.0,3.0,0.0,0.0,0.0,1.0,0.0,138.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0]|Online|0.0 |[-0.6158668250746622,0.8492147669420864,-0.23334794186742447] |[0.14719153609284713,0.6370297114665976,0.2157787524405553] |1.0 |Freebook |
+-------------------------------------------------------------------------+------+------------+---------------------------------------------------------------+------------------------------------------------------------+----------+--------------+
only showing top 5 rows

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [0]:
class_temp = predictions.select("label").groupBy("label")\
                        .count().sort('count', ascending=False).toPandas()
class_temp = class_temp["label"].values.tolist()
class_names = map(str, class_temp)
# # # print(class_name)


In [0]:
from sklearn.metrics import confusion_matrix
y_true = prediction.select("label")
y_true = y_true.toPandas()

y_pred = prediction.select("predictedLabel")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)
cnf_matrix

Out[25]: array([[1169, 117, 199],
 [ 345, 841, 320],
 [ 352, 410, 700]])

In [0]:
#plt.figure()
#plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
#                      title='Normalized confusion matrix')
#
#plt.show()

test

In [0]:
#data preprocessen 
df_test  = spark.read.format('delta').load(
 "dbfs:/mnt/bastore/modelling/recommendations/data_for_model/test")
df_test1 = df_test.na.fill(0)

In [0]:
df_test

Out[28]:

ConsumerID,FK_Time,FK_Time_Date,FK_Time_Insert,FK_Consumer,Country,Seniority_max,Product_sum,Upsell_avg,SubBreak_sum,RetIntention_sum,Retention_sum,Web_sum_Y1,Web_sum_Y2,Web_sum_DY1Y2,PAS_sum_Y1,PAS_sum_Y2,PAS_sum_DY1Y2,Freebook_sum,Newsletter_sum,Payments_sum,Discount_sum,Payments_sum_Y4,Payments_change,Web_sum_Y1_Fin,Web_sum_Y1_PortF,FB_Heal,FB_Fin,Fline,ExFline,Seniority_ExFline,Web_sum_Y1_Hline,Web_sum_Y2_Hline,Web_sum_DY1Y2_Hline,Newsletter_sum_Hline,Hline,Seniority_Fline,Newsletter_sum_Fline,Seniority_Hline,Seniority_ExHline,ExHline,Telemarketing_sum,ComMail_sum,Survey_sum,CampaignType_Fline,FK_Time2,webv_bool_inf,freeb_bool_inf,pas_Bool_inf,ServiceGroup
null,20231201,null,2024-01-14 18:33:...,481,BE,375,2,-225,0,0,0,184,null,null,0,null,null,2,0,273,0,null,10,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,6,1,null,null,null,null,20231201,1,1,0,Ni
null,20231201,null,2024-01-14 18:33:...,673,BE,578,2,0,0,0,0,195,null,null,0,null,null,1,0,273,0,null,10,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,0,0,null,null,null,null,20231201,1,1,1,ALL
null,20231201,null,2024-01-14 18:33:...,1005,BE,373,2,0,0,0,0,0,null,null,0,null,null,0,0,270,0,null,8,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,0,0,null,null,null,null,20231201,1,1,0,Ni
null,20231201,null,2024-01-14 18:33:...,1417,BE,74,2,0,0,0,0,0,null,null,3,null,null,1,0,273,0,null,10,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,0,0,null,null,null,null,20231201,1,1,1,ALL
null,20231201,null,2024-01-14 18:33:...,1829,BE,247,1,0,0,0,0,85,null,null,8,null,null,1,0,173,0,null,4,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,0,0,null,null,null,null,20231201,1,1,1,ALL
null,20231201,null,2024-01-14 18:33:...,2096,BE,198,4,0,0,2,0,0,null,null,0,null,null,0,0,426,0,null,8,null,null,null,null,0,1,2,null,null,null,null,1,0,null,63,0,0,null,null,null,null,20231201,1,0,1,Ni
null,20231201,null,2024-01-14 18:33:...,2156,BE,567,1,0,0,0,0,0,null,null,0,null,null,0,0,177,0,null,11,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,0,0,null,null,null,null,20231201,1,0,1,Ni
null,20231201,null,2024-01-14 18:33:...,2542,BE,575,1,-41,0,0,0,328,null,null,15,null,null,0,0,255,0,null,-7,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,0,0,null,null,null,null,20231201,1,1,1,ALL
null,20231201,null,2024-01-14 18:33:...,2748,BE,672,4,0,0,1,0,410,null,null,0,null,null,2,0,386,0,null,67,null,null,null,null,0,0,0,null,null,null,null,0,0,null,0,0,0,null,null,null,null,20231201,1,1,1,ALL
null,20231201,null,2024-01-14 18:33:...,2811,BE,77,4,0,0,0,2,0,null,null,0,null,null,5,0,436,0,null,1,null,null,null,null,0,0,0,null,null,null,null,1,0,null,77,0,0,null,null,null,null,20231201,1,1,0,Ni


In [0]:
from pyspark.sql.types import IntegerType
df_test2 = featureIndexer.transform(df_test1).drop('Country')

df_test3 = df_test2.select( "CountryIndex","Seniority_max", "Product_sum", "Upsell_avg", "SubBreak_sum", "Retention_sum", "RetIntention_sum", "Discount_sum", "Payments_sum", "Fline", "Hline", "ExHline", "ExFline", "Seniority_Hline", "Seniority_Fline", "Seniority_ExFline", "Seniority_ExHline",  col("FK_Consumer").cast(IntegerType()), col("FK_Time").cast(IntegerType()))


In [0]:

def transDataTest(data): 
  return data.rdd.map(lambda r: [Vectors.dense(r[0:17]),r[17], r[18]]).toDF(['features', 'FK_Consumer', 'FK_Time'])
Test =transDataTest(df_test3)
Test_pred = model.transform(Test)
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)
Test_pred1 = labelConverter.transform(Test_pred)
Test_pred1.show(10, False)

+----------------------------------------------------------------------------+-----------+--------+-----------------------------------------------------------+------------------------------------------------------------------+----------+--------------+
features |FK_Consumer|FK_Time |rawPrediction |probability |prediction|predictedLabel|
+----------------------------------------------------------------------------+-----------+--------+-----------------------------------------------------------+------------------------------------------------------------------+----------+--------------+
[1.0,565.0,2.0,0.0,0.0,0.0,0.0,0.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |374 |20231201|[-1.7311409864039902,-52.01812405240559,53.749265038809575]|[8.038329593778937E-25,1.1636076086563667E-46,1.0] |2.0 |PAS |
[1.0,142.0,3.0,0.0,0.0,0.0,0.0,0.0,347.0,0.0,0.0,0.0,1.0,0.0,0.0,65.0,0.0] |547 |20231201|[-57.39756797361284,-32.18747078084156,89.5850387544544] |[1.4664422417389942E-64,1.3027799356906845E-53,1.0] |2.0 |PAS |
[1.0,672.0,2.0,0.0,0.0,0.0,1.0,14.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |1369 |20231201|[8.261687164756037,-58.28317033991415,50.0214831751581] |[7.310566420649564E-19,9.20209291222251E-48,1.0] |2.0 |PAS |
[1.0,168.0,3.0,133.0,0.0,0.0,7.0,0.0,184.0,0.0,1.0,0.0,1.0,8.0,0.0,28.0,0.0]|1520 |20231201|[-31.111023046290295,-5.321838001956287,36.432861048246586]|[4.635150622643357E-30,7.347923208886296E-19,1.0] |2.0 |PAS |
[1.0,131.0,2.0,-66.0,0.0,1.0,0.0,0.0,296.0,0.0,0.0,0.0,1.0,0.0,0.0,38.0,0.0]|2089 |20231201|[-42.840877129853325,-36.83244735293642,79.67332448278974] |[6.205235734778458E-54,2.5245628195644227E-51,1.0] |2.0 |PAS |
[1.0,211.0,3.0,0.0,0.0,0.0,0.0,0.0,285.0,0.0,1.0,0.0,0.0,144.0,0.0,0.0,0.0] |2403 |20231201|[-22.371935988357187,-37.03783840790392,59.409774396261106]|[3.038466548043482E-36,1.2981771674172002E-42,1.0] |2.0 |PAS |
[1.0,672.0,1.0,0.0,0.0,0.0,1.0,0.0,179.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |2861 |20231201|[27.254583480942866,-52.28717783943729,25.032594358494432] |[0.9022068357632115,2.5748865043052255E-35,0.09779316423678847] |0.0 |Online |
[1.0,672.0,4.0,0.0,0.0,0.0,0.0,0.0,382.0,0.0,1.0,0.0,0.0,261.0,0.0,0.0,0.0] |3066 |20231201|[14.045399384148098,-75.03555848972617,60.99015910557805] |[4.094019422825061E-21,8.409718356790331E-60,1.0] |2.0 |PAS |
[1.0,672.0,2.0,0.0,0.0,0.0,2.0,14.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |3300 |20231201|[22.60309966397966,-53.70255550928685,31.099455845307197] |[2.041694295934313E-4,1.4820598281200468E-37,0.9997958305704064] |2.0 |PAS |
[1.0,576.0,1.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |3374 |20231201|[37.30281735375231,-40.45419360861451,3.151376254862196] |[0.9999999999999984,1.7004317670475022E-34,1.4730503163888897E-15]|0.0 |Online |
+----------------------------------------------------------------------------+-----------+--------+-----------------------------------------------------------+------------------------------------------------------------------+----------+--------------+
only showing top 10 rows

In [0]:
from pyspark.ml.functions import vector_to_array

df_result = Test_pred1.select(col('probability'), col('FK_Consumer').cast('int'), col('FK_Time').cast('int'), col('predictedLabel'))
df_result
df_servicegroup = df_test.select(col('serviceGroup'), col('FK_Consumer'), col('FK_Time'))
df_result1 = df_result.join(df_servicegroup, ["FK_Consumer","FK_Time"])
df_result2 = (df_result1.withColumn("xs", vector_to_array("probability")))
df_result3 = df_result2.select(["FK_Consumer", "FK_Time","serviceGroup", "predictedLabel"]+[col("xs")[i] for i in range(3)])\
  .withColumnRenamed("xs[0]", "Freebook") \
  .withColumnRenamed("xs[1]", "PAS") \
  .withColumnRenamed("xs[2]", "Online") 



In [0]:
df_result3.write.format("delta").mode("overwrite").save("dbfs:/mnt/bastore/modelling/recommendations/results_from_model/results_recoom")